# Load and Preview Data

This cell loads the Excel file into a pandas DataFrame and shows basic information about the dataset.


In [1]:
import pandas as pd

# Set pandas display options to avoid truncation and line breaks
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', 1000)         # Set a wider display width
pd.set_option('display.colheader_justify', 'left')  # Justify column headers to the left

df = pd.read_excel('./shared/Ask A Manager Salary Survey 2021 (Responses).xlsx', engine='openpyxl')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28085 entries, 0 to 28084
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype         
---  ------                                                                                                                                                                                                                                --------------  -----         
 0   Timestamp                                                                                                                                                                                                                             28085 non-null  datetime64[ns]
 1   How old are you?                                                                        

## Helper Functions to display data

In [2]:
def display_unique_values(df, column_name):
    """
    Displays the unique values of a specified column in a DataFrame, ordered alphabetically or numerically.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column_name (str): The column name whose unique values you want to display.
    
    Returns:
    None: Prints the unique values in the specified column, sorted in ascending order.
    """
    if column_name in df.columns:
        unique_values = sorted(df[column_name].unique())
        print(f"Unique values in '{column_name}' (sorted):")
        for value in unique_values:
            print(value)
    else:
        print(f"Column '{column_name}' does not exist in the DataFrame.")


In [3]:
def display_value_counts(df, column_name):
    """
    Displays the count of unique values in a specified column of a DataFrame,
    sorted in descending order.

    Parameters:
    df (pd.DataFrame): The DataFrame to analyze.
    column_name (str): The column to count unique values from.

    Returns:
    None
    """
    if column_name in df.columns:
        value_counts = df[column_name].value_counts().sort_values(ascending=False)
        print(f"Unique value counts for column '{column_name}':\n")
        print(value_counts)
    else:
        print(f"Column '{column_name}' not found in the DataFrame.")

# Rename Columns for Clarity

I renamed the columns in the DataFrame because the original titles from the Excel file were too long and hard to work with in code. By shortening them, I make the DataFrame easier to manipulate and reference. For instance, long titles like "What is your job title?" are now simply "Job Title," which is cleaner and more manageable for analysis. 
This step also improves code readability.



In [4]:
df.columns = ['Timestamp', 'Age Group', 'Industry', 'Job Title', 'Job Title Additional Context',
              'Annual Salary', 'Additional Compensation', 
              'Currency','Other Currency', 'Income Additional Context', 'Country', 'State', 'City', 
              'Total Experience', 'Field Experience', 
              'Education Level', 'Gender', 'Race']

df.head()

,Timestamp,Age Group,Industry,Job Title,Job Title Additional Context,Annual Salary,Additional Compensation,Currency,Other Currency,Income Additional Context,Country,State,City,Total Experience,Field Experience,Education Level,Gender,Race
0,2021-04-27 11:02:09.743,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27 11:02:21.562,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27 11:02:38.125,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27 11:02:40.643,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27 11:02:41.793,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


# Analyzing Missing Values

In this step, I wanted to understand the extent of missing data in the dataset. I calculated both the count and percentage of missing values per column to help me decide which columns may need further cleaning or handling before moving forward with analysis.

After running the cell, I got a clearer understanding of which columns have substantial missing data, and which ones seem reliable for analysis.

## Columns to Trust
The following columns have no missing data and can be trusted for immediate use:
- **Timestamp**
- **Age Group**
- **Annual Salary**
- **Currency**
- **Total Experience**
- **Field Experience**
- **Country**

## Rows to drop
For the purposes of this analysis, I will drop rows that have missing values in the **Job Title** and **Industry** columns, as these fields are crucial for understanding salary data and making meaningful comparisons.

## Columns to Ignore
Certain columns have a very high percentage of missing data, indicating that they might be less relevant for the analysis or could be safely ignored. These columns include:
- **Job Title Additional Context**
- **Other Currency**
- **Income Additional Context**

Since the missing values in these columns are so high, they may not provide meaningful insights and can be ignored during further analysis.

## Columns to Handle Later
Certain columns have moderate levels of missing data and should be addressed if future metrics or analyses depend on them. For now, I'll skip handling them, but this could be an enhancement for future analysis:
- **Education Level**
- **Gender**
- **Race**
- **State**
- **City**

These columns might require attention if relevant metrics are involved later.


In [5]:
missing_values = df.isnull().sum()

missing_percentage = (df.isnull().sum() / len(df)) * 100

print(missing_values)
print(missing_percentage)

Timestamp                           0
Age Group                           0
Industry                           74
Job Title                           1
Job Title Additional Context    20818
Annual Salary                       0
Additional Compensation          7305
Currency                            0
Other Currency                  27878
Income Additional Context       25041
Country                             0
State                            5026
City                               82
Total Experience                    0
Field Experience                    0
Education Level                   222
Gender                            171
Race                              177
dtype: int64
Timestamp                        0.000000
Age Group                        0.000000
Industry                         0.263486
Job Title                        0.003561
Job Title Additional Context    74.124978
Annual Salary                    0.000000
Additional Compensation         26.010326
Currency 

# Data Cleaning Process and Reasoning

In this analysis, I performed several data cleaning steps to prepare the dataset for further exploration and ensure that it's suitable for analysis. Here’s a breakdown of what was done and the reasoning behind each step:

#### Step 1: Dropping Columns with High Missing Percentages
I dropped the columns **Job Title Additional Context**, **Income Additional Context**, and **Other Currency** because they had a very high percentage of missing values (above 70%). Keeping these columns would likely introduce noise into the analysis, and since they are not essential for the core questions being addressed, I opted to remove them.

#### Step 2: Handling Missing Values in Numeric Columns
For the **Additional Compensation** column, I filled the missing values with `0.0`. The assumption here is that if no value was provided, the respondent did not receive any additional compensation. This ensures that the analysis is not skewed by missing data.

#### Step 3: Filling Missing Values in State and City Columns
For the **State** and **City** columns, I filled the missing values with 'Unknown'. This approach prevents any data loss while still distinguishing between known and unknown values. Using 'Unknown' is a common best practice when there is a need to preserve data but the true value is unavailable.

#### Step 4: Imputing Categorical Columns with the Most Frequent Value
I imputed missing values in the **Education Level**, **Gender**, and **Race** columns using the mode (the most frequent value). This method ensures that these key categorical variables remain usable while minimizing any potential bias introduced by missing data.

### Enhancement: Rather than arbitrarily choosing a placeholder for missing categorical data, imputing with the mode ensures consistency and prevents introducing outliers.

#### Step 5: Final Check on Missing Values
After all the imputations and cleaning, I performed a final check to confirm that no missing values remain in the dataset. This step ensures that the dataset is now fully prepared for analysis.

#### Step 6: Final Data Structure and Initial Review
Finally, I reviewed the structure of the cleaned dataset and displayed the first few rows to confirm that everything is in order before proceeding with any analysis.


In [6]:
# Define columns to drop and constants for imputation
COLUMNS_TO_DROP = ['Job Title Additional Context', 'Income Additional Context', 'Other Currency']
NUMERIC_FILL_VALUE = 0.0
CATEGORICAL_FILL_VALUE = 'Unknown'

# Function to fill missing values for categorical columns with mode
def fill_missing_with_mode(df, columns):
    for col in columns:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df

# Drop columns with high missing percentage
df_clean = df.drop(COLUMNS_TO_DROP, axis=1, inplace=False)
# Drop columns with negative salary
df_clean = df_clean[df_clean['Annual Salary'] >= 0]

# Fill missing values for numeric columns
df_clean['Additional Compensation'] = df_clean['Additional Compensation'].fillna(NUMERIC_FILL_VALUE)

# Fill missing values for State and City with 'Unknown'
df_clean['State'] = df_clean['State'].fillna(CATEGORICAL_FILL_VALUE)
df_clean['City'] = df_clean['City'].fillna(CATEGORICAL_FILL_VALUE)

# Impute categorical columns with the most frequent value (mode)
categorical_columns = ['Industry', 'Job Title', 'Education Level', 'Gender', 'Race']
df_clean = fill_missing_with_mode(df_clean, categorical_columns)

# Final check on missing values after cleaning
print(df_clean.isnull().sum())

# Display the cleaned DataFrame structure and first few rows
df_clean.info()
df_clean.head()

Timestamp                  0
Age Group                  0
Industry                   0
Job Title                  0
Annual Salary              0
Additional Compensation    0
Currency                   0
Country                    0
State                      0
City                       0
Total Experience           0
Field Experience           0
Education Level            0
Gender                     0
Race                       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28085 entries, 0 to 28084
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Timestamp                28085 non-null  datetime64[ns]
 1   Age Group                28085 non-null  object        
 2   Industry                 28085 non-null  object        
 3   Job Title                28085 non-null  object        
 4   Annual Salary            28085 non-null  int64         
 5   Additional 

,Timestamp,Age Group,Industry,Job Title,Annual Salary,Additional Compensation,Currency,Country,State,City,Total Experience,Field Experience,Education Level,Gender,Race
0,2021-04-27 11:02:09.743,25-34,Education (Higher Education),Research and Instruction Librarian,55000,0.0,USD,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27 11:02:21.562,25-34,Computing or Tech,Change & Internal Communications Manager,54600,4000.0,GBP,United Kingdom,Unknown,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27 11:02:38.125,25-34,"Accounting, Banking & Finance",Marketing Specialist,34000,0.0,USD,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27 11:02:40.643,25-34,Nonprofits,Program Manager,62000,3000.0,USD,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27 11:02:41.793,25-34,"Accounting, Banking & Finance",Accounting Manager,60000,7000.0,USD,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


# Column Exploration

### Column Race

In this step, I wanted to get an overview of how many unique values each column in the dataset contains. To do this, I wrote a simple command to display the number of distinct values per column.
With this output, I found it interesting to see that the 'Race' column has a surprisingly high number of distinct combinations (51 unique values). I decided to explore further by printing all the unique values in this column to understand what these combinations looked like:

In [7]:
# Display the number of distinct values in each column
distinct_values_count = df_clean.nunique()

# Display the result
print(distinct_values_count)

# Show all unique values in the 'Race' column
unique_race_values = df_clean['Race'].unique()

# Display the unique values
print(unique_race_values)

Timestamp                  28080
Age Group                      7
Industry                    1220
Job Title                  14360
Annual Salary               3668
Additional Compensation      847
Currency                      11
Country                      379
State                        136
City                        4835
Total Experience               8
Field Experience               8
Education Level                6
Gender                         5
Race                          51
dtype: int64
['White' 'Hispanic, Latino, or Spanish origin, White'
 'Asian or Asian American, White' 'Asian or Asian American'
 'Another option not listed here or prefer not to answer'
 'Hispanic, Latino, or Spanish origin'
 'Middle Eastern or Northern African'
 'Hispanic, Latino, or Spanish origin, Middle Eastern or Northern African, White'
 'Black or African American' 'Black or African American, White'
 'Black or African American, Hispanic, Latino, or Spanish origin, White'
 'Native American or Ala

**Comments:**

While this exploration yielded a variety of combinations involving multiple racial identities and options, I realized that analyzing these combinations requires an in-depth understanding of racial identity classifications, which is not my area of expertise. Given the complexity and nuanced nature of these combinations, I've decided to skip any analysis involving the 'Race' column for now, as I don't feel confident in using it as a reliable metric for this dataset.

Because of this, we won't be using 'Race' in any further analysis. This decision limits the scope of our insights regarding racial diversity, but I believe it's important to avoid misinterpreting or misrepresenting data.

### Column Country

I found this API that provides a list of country names, so I decided to use it to validate the values in the `Country` column of my dataset. I'll use this external source to ensure that the country names in my dataset are valid. For now, I'll be validating the country names only, focusing on whether the names match those provided by the API.

#### Download the list
Get a valid list from an url

In [8]:
import requests

# Function to fetch and extract country names from the API
def fetch_country_names(api_url):
    # Make the GET request
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Check for HTTP request errors
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return []

    # Extract the 'data' field (which contains the list of countries)
    try:
        data = response.json()
        countries_data = data.get('data', [])
    except ValueError as e:
        print(f"Error parsing JSON response: {e}")
        return []

    # Extract the 'name' field from each country into a single list
    return [country['name'] for country in countries_data if 'name' in country]

# Define the API URL and fetch the country names
url = "https://countriesnow.space/api/v0.1/countries/positions"
country_names_list = fetch_country_names(url)

# Display the list of country names
print(country_names_list)

['Afghanistan', 'Albania', 'Algeria', 'AmericanSamoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', 'Ivory Coast', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Isl

### Remove accents using `unidecode`
I started by using the `unidecode` library to remove any accents or diacritics from the country names. This is important because accented characters can lead to inconsistencies when comparing or matching text, especially across different datasets.

### Normalize country names by removing special characters
Next, I removed any special characters (like punctuation). This ensures that variations like "USA" vs "Usa" or "U.K." vs "UK" are treated consistently and won’t be considered different entities during the analysis.

### Strip leading and trailing spaces
Some country names might have had extra spaces either at the beginning or end. I applied `strip()` to remove those unnecessary spaces, ensuring cleaner data without unintended formatting issues.

### Use regex to handle various cases of 'USA' and 'US'
I used regular expressions to handle common variations of "United States" such as "USA", "U.S.", or "America". This step ensures that all these variations are unified into a single consistent label, "United States", which makes analysis much easier and avoids duplication.

### Handle 'UK', 'Great Britain', 'Britain', and 'England' replacements in compound names
Similarly, I handled multiple ways people might refer to the United Kingdom. Whether it’s "UK", "Great Britain", "Britain", or even "England" and "Scotland", I standardized these variations into a single label, "United Kingdom". This step ensures that all references to this region are treated consistently in the data.


In [9]:
from fuzzywuzzy import process
from unidecode import unidecode

# Ensure all country names in the dataframe are strings
df_clean['Country'] = df_clean['Country'].astype(str)

# Remove accents using unidecode and normalize country names to lowercase
df_clean['Country'] = df_clean['Country'].apply(lambda x: unidecode(x)).str.replace(r'[^\w\s]', '', regex=True).str.strip()

# Handle various cases of 'USA', 'US', 'Great Britain', 'UK', 'England', and 'Scotland'
df_clean['Country'] = df_clean['Country'].str.replace(r'\b(us|usa|usaa|the us|u\.s\.a|u\.s\.|america|usa )\b', 'United States', regex=True, case=False)
df_clean['Country'] = df_clean['Country'].str.replace(r'\buk\b|\bgreat britain\b|\bbritain\b|\bengland\b|\benglanduk\b|\bscotland\b', 'United Kingdom', case=False, regex=True)

# Remove rows where the 'Country' has more than 6 words
df_clean = df_clean[df_clean['Country'].apply(lambda x: len(x.split()) <= 6)]
df_clean = df_clean[df_clean['Industry'].apply(lambda x: len(x.split()) <= 6)]

# Lowercase the valid country names for comparison
#country_names_list_lower = [unidecode(name).lower() for name in country_names_list]
country_names_list_lower = country_names_list

### Check for invalid country names
Comparing Country column with api list to find wrong/bad values

In [10]:
# Function to validate and print distinct invalid countries
def validate_countries():
    # Find invalid countries
    invalid_countries = df_clean[~df_clean['Country'].isin(country_names_list_lower)]
    
    # Print distinct and ordered invalid countries
    distinct_invalid_countries = sorted([x for x in invalid_countries['Country'].unique() if isinstance(x, str)])
    print(f"Distinct invalid countries: {distinct_invalid_countries}")
    return invalid_countries

# Validate countries before fuzzy matching
invalid_countries = validate_countries()

Distinct invalid countries: ['', '217584year is deducted for benefits', 'Africa', 'Aotearoa New Zealand', 'Australi', 'Australian', 'Brasil', 'Burma', 'CANADA', 'California', 'Can', 'Canad', 'Canada Ottawa ontario', 'Canada and United States', 'Canadw', 'Canda', 'Catalonia', 'Contracts', 'Cote dIvoire', 'Csnada', 'Currently finance', 'Czech republic', 'Czechia', 'Danmark', 'Englang', 'FRANCE', 'Global', 'Hartford', 'INDIA', 'IS', 'ISA', 'International', 'Italy South', 'Japan United States Gov position', 'Jersey Channel islands', 'LOUTRELAND', 'London', 'Luxemburg', 'Mainland China', 'NIGERIA', 'NL', 'NZ', 'Nederland', 'New Zealand Aotearoa', 'New zealand', 'Northern Ireland', 'Northern Ireland United Kingdom', 'Policy', 'Remote', 'Remote philippines', 'SWITZERLAND', 'San Francisco', 'South africa', 'Sri lanka', 'The Bahamas', 'The Netherlands', 'The United States', 'The netherlands', 'U S', 'UA', 'UAE', 'UNITED STATES', 'UNited States', 'USAB', 'USD', 'USS', 'UXZ', 'Uniited States', 'U

**Comments:**

With the result printed, I noticed there are some spelling mistakes or variations in country names, like 'csnada' instead of 'Canada', 'danmark' instead of 'Denmark', and 'united kingdomk' instead of 'United Kingdom'. To address these issues, I decided to use **fuzzy matching**. Fuzzy matching is a method that helps find the closest match between two strings, which is particularly useful when dealing with typos, spelling errors, or different formats. By applying fuzzy matching, I aim to correct these mistakes and standardize the country names in the dataset.


In [11]:
# Apply fuzzy matching only to the rows with invalid country names
def fuzzy_match_country(country):    
    if len(country) >= 4:        
        match, score = process.extractOne(country, country_names_list_lower)                
        if score >= 80:  # Set a threshold for fuzzy matching accuracy
            print(f"Matched '{country}' to '{match}' with score {score}")
            return match
        else:
            return 'Unknown'
    return 'Unknown'

# Apply fuzzy matching to distinct invalid country names
distinct_invalid_countries = sorted([x for x in invalid_countries['Country'].unique() if isinstance(x, str)])

# Create fuzzy matched dictionary
fuzzy_matched_dict = {country: fuzzy_match_country(country) for country in distinct_invalid_countries}

Matched 'Africa' to 'Central African Republic' with score 90
Matched 'Aotearoa New Zealand' to 'New Zealand' with score 90
Matched 'Australi' to 'Australia' with score 94
Matched 'Australian' to 'Australia' with score 95
Matched 'Brasil' to 'Brazil' with score 83
Matched 'CANADA' to 'Canada' with score 100
Matched 'Canad' to 'Canada' with score 91
Matched 'Canada Ottawa ontario' to 'Canada' with score 90
Matched 'Canada and United States' to 'Canada' with score 90
Matched 'Canadw' to 'Canada' with score 83
Matched 'Canda' to 'Canada' with score 91
Matched 'Csnada' to 'Canada' with score 83
Matched 'Czech republic' to 'Czech Republic' with score 100
Matched 'Danmark' to 'Denmark' with score 86
Matched 'FRANCE' to 'France' with score 100
Matched 'INDIA' to 'India' with score 100
Matched 'Italy South' to 'Italy' with score 90
Matched 'Japan United States Gov position' to 'Japan' with score 90
Matched 'Jersey Channel islands' to 'Jersey' with score 90
Matched 'Luxemburg' to 'Luxembourg' wi

### Remove invalid countries from dataset

In this step, I updated the invalid country names in the dataframe by applying the fuzzy matching results. This was done using the `fuzzy_matched_dict` to map the incorrect country names to their correct counterparts. The reason for this is to ensure that the dataset contains clean and accurate country information.

I found the number of 'Unknown' countries and printed it before removing them of my dataset.

Finally, I re-ran the validation function to ensure there are no remaining invalid country names after this step.

63 rows removed from dataset as result.


In [12]:
# Update invalid countries in the dataframe using the fuzzy matched dictionary
df_clean.loc[invalid_countries.index, 'Country'] = df_clean['Country'].map(fuzzy_matched_dict)

# Count rows where 'Country' is 'Unknown'
unknown_country_count = df_clean[df_clean['Country'] == 'Unknown'].shape[0]

print(f"Number of rows with 'Unknown' Contry name: {unknown_country_count}. They will be deleted from dataset")

df_clean = df_clean[df_clean['Country'] != 'Unknown']

# Re-validate countries after applying fuzzy matching
invalid_countries = validate_countries()

Number of rows with 'Unknown' Contry name: 63. They will be deleted from dataset
Distinct invalid countries: []


### Column Industry

In the Industry column, we have over 1200 unique values. To simplify the dataset and make it more manageable for analysis, we decided to create a new "Industry Category" column. This new column groups similar industry values together, significantly reducing the number of unique categories while preserving the integrity of the data. This helps in making the data more structured and easier to work with for future analysis or reporting.

Firstly, let's save an unique industry list into a pandas dataframe

In [13]:
unique_ind_values = sorted(df_clean['Industry'].unique())
df_industries = pd.DataFrame({'Industry': unique_ind_values})

### Industry Clustering Process

In this analysis, we processed the `Industry` column to reduce the number of unique categories. Here's the approach:

1. **Text Preprocessing**: I started by normalizing the text. This involved converting everything to lowercase, removing special characters and redundant spaces, and applying some specific rules like replacing "science/research" patterns with "science research." Additionally, I used lemmatization to reduce words to their root form, improving consistency across similar terms.
   
2. **Embedding Generation**: I utilized the paraphrase-MiniLM-L6-v2 model to encode the processed industry names into embeddings. These embeddings help capture the semantic meaning behind the industry names, allowing for more effective clustering of similar terms.

3. **Clustering**: I experimented with different distance metrics (`cosine`, `euclidean`, `manhattan`) to see how each method grouped the industry names based on their similarity.

4. **Results**: Let's check below the resulf of each metric:

- Metric: euclidean - Unique Representative Categories: 855
- Metric: manhattan - Unique Representative Categories: 855
- Metric: cosine - Unique Representative Categories: 388

Let's use 'cosine' metric from now on.

In [14]:
import re
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np

nltk.download('wordnet', quiet=True)

def preprocess_text(text, stop_words, lemmatizer):
    """
    Preprocesses the input text by normalizing case, removing special characters, 
    applying custom rules, and performing lemmatization.

    Args:
        text (str): The raw input text (industry name).
        stop_words (set): A set of stopwords to remove.
        lemmatizer (WordNetLemmatizer): NLTK lemmatizer to reduce words to their base forms.

    Returns:
        str: Preprocessed text.
    """
    text = text.lower()
    text = text.replace('/', ' ')
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'scien\w*\sresearch\w*.*', 'science research', text)
    text = re.sub(r'\breal\s*estate\w*.*', 'Real Estate', text)
    
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

def encode_industry_names(industry_names, model):
    """
    Encodes the industry names into embeddings using a pre-trained sentence transformer model.

    Args:
        industry_names (pd.Series): A pandas Series of industry names.
        model (SentenceTransformer): Pre-trained SentenceTransformer model.

    Returns:
        np.ndarray: The embeddings for the industry names.
    """
    return model.encode(industry_names.values)

def compute_cosine_similarity(embeddings, single_embedding=None):
    if single_embedding is not None:
        # If comparing a single embedding against a set of embeddings
        return cosine_similarity(single_embedding, embeddings)
    else:
        # Compute pairwise cosine similarity for all embeddings
        return 1 - cosine_similarity(embeddings)

def perform_clustering(embeddings, df, metric, distance_threshold=0.4):
    """
    Performs Agglomerative Clustering on the industry embeddings and assigns cluster labels.

    Args:
        embeddings (np.ndarray): Encoded industry embeddings.
        df (pd.DataFrame): The original DataFrame of industries.
        metric (str): The metric to use for clustering (e.g., 'cosine', 'euclidean', 'manhattan').
        distance_threshold (float): The distance threshold for clustering.

    Returns:
        pd.DataFrame: A DataFrame with cluster labels and representative categories.
    """
    df_copy = df.copy()

    if metric == 'cosine':
        cosine_distances = compute_cosine_similarity(embeddings)
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold, metric='precomputed', linkage='complete')
        clusters = clustering.fit_predict(cosine_distances)
    else:
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold, metric=metric, linkage='complete')
        clusters = clustering.fit_predict(embeddings)
    
    # Assign clusters to the copy of the DataFrame
    df_copy['Cluster'] = clusters
    
    # Assign representative categories based on the first item in each cluster
    df_copy['Industry Category'] = df_copy.groupby('Cluster')['Industry'].transform('first')
    
    # Calculate the number of unique categories
    unique_categories_count = df_copy['Industry Category'].nunique()
    
    # Display the metric and the number of unique categories
    print(f"Metric: {metric} - Unique Representative Categories: {unique_categories_count}")
    
    return df_copy

def run_clustering(df_industries, model, metrics):
    """
    Runs the clustering process for each provided metric and displays the results.

    Args:
        df_industries (pd.DataFrame): The original DataFrame with industry names.
        model (SentenceTransformer): Pre-trained SentenceTransformer model.
        metrics (list): List of metrics to use for clustering.

    Returns:
        None
    """
    # Preprocess industry names
    stop_words = sklearn_stopwords
    lemmatizer = WordNetLemmatizer()
    df_industries['Processed_Industry'] = df_industries['Industry'].apply(preprocess_text, args=(stop_words, lemmatizer))

    # Encode industry names
    industry_embeddings = encode_industry_names(df_industries['Processed_Industry'], model)

    # Run clustering for each metric
    clustered_df = None
    for metric in metrics:
        clustered_df = perform_clustering(industry_embeddings, df_industries, metric)        
    
    # Return the final clustered DataFrame
    return clustered_df

# Load pre-trained model and define metrics
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
metrics = ['euclidean', 'manhattan', 'cosine']

# Run clustering
x = run_clustering(df_industries, model, metrics)

/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Metric: euclidean - Unique Representative Categories: 855
Metric: manhattan - Unique Representative Categories: 855
Metric: cosine - Unique Representative Categories: 388


**Comments:**

In the previous cell, I applied some specific rules for handling patterns like 'science research' and 'real estate.' The cosine similarity method wasn't sufficient to correctly group these terms, so I manually introduced mappings to ensure better categorization:

- `r'scien\w*\sresearch\w*.*'` was mapped to 'science research'
- `r'\breal\s*estate\w*.*'` was mapped to 'Real Estate'

These adjustments allowed for more accurate grouping of similar industry names that were not effectively handled by the cosine similarity alone.


In [15]:
metrics = ['cosine']

df_best_metric = run_clustering(df_industries, model, metrics)

df_clean = pd.merge(df_clean, df_best_metric[['Industry', 'Industry Category']], on='Industry', how='left')

df_clean.head(10)

Metric: cosine - Unique Representative Categories: 388


,Timestamp,Age Group,Industry,Job Title,Annual Salary,Additional Compensation,Currency,Country,State,City,Total Experience,Field Experience,Education Level,Gender,Race,Industry Category
0,2021-04-27 11:02:09.743,25-34,Education (Higher Education),Research and Instruction Librarian,55000,0.0,USD,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,Adult education
1,2021-04-27 11:02:21.562,25-34,Computing or Tech,Change & Internal Communications Manager,54600,4000.0,GBP,United Kingdom,Unknown,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,Computing or Tech
2,2021-04-27 11:02:38.125,25-34,"Accounting, Banking & Finance",Marketing Specialist,34000,0.0,USD,United States,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,"Accounting, Banking & Finance"
3,2021-04-27 11:02:40.643,25-34,Nonprofits,Program Manager,62000,3000.0,USD,United States,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,Nonprofit - legal department
4,2021-04-27 11:02:41.793,25-34,"Accounting, Banking & Finance",Accounting Manager,60000,7000.0,USD,United States,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,"Accounting, Banking & Finance"
5,2021-04-27 11:02:45.571,25-34,Education (Higher Education),Scholarly Publishing Librarian,62000,0.0,USD,United States,New Hampshire,Hanover,8 - 10 years,2 - 4 years,Master's degree,Man,White,Adult education
6,2021-04-27 11:02:50.507,25-34,Publishing,Publishing Assistant,33000,2000.0,USD,United States,South Carolina,Columbia,2 - 4 years,2 - 4 years,College degree,Woman,White,Academic Publishing
7,2021-04-27 11:02:59.927,25-34,Education (Primary/Secondary),Librarian,50000,0.0,USD,United States,Arizona,Yuma,5-7 years,5-7 years,Master's degree,Man,White,Adult education
8,2021-04-27 11:03:01.045,45-54,Computing or Tech,Systems Analyst,112000,10000.0,USD,United States,Missouri,St. Louis,21 - 30 years,21 - 30 years,College degree,Woman,White,Computing or Tech
9,2021-04-27 11:03:01.699,35-44,"Accounting, Banking & Finance",Senior Accountant,45000,0.0,USD,United States,Florida,Palm Coast,21 - 30 years,21 - 30 years,College degree,Woman,"Hispanic, Latino, or Spanish origin, White","Accounting, Banking & Finance"


### QA

Check below new 'Industry Category' has 388 unique values but it is not good enough.

In [16]:
# Display the number of distinct values in each column
distinct_values_count = df_clean.nunique()

print(distinct_values_count)

Timestamp                  27966
Age Group                      7
Industry                    1185
Job Title                  14302
Annual Salary               3654
Additional Compensation      844
Currency                      11
Country                       97
State                        134
City                        4798
Total Experience               8
Field Experience               8
Education Level                6
Gender                         5
Race                          50
Industry Category            388
dtype: int64


### Industry Category Enhancement

There are some categories with just 1 or 2 rows. I want to realocate them to another existing category. Let's do it with AI to avoid manual mapping.

Industry categories have 162 unique values as result.

In [17]:
# Load the pre-trained language model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to reallocate categories that appear fewer than min_occurrence times
def reallocate_with_AI(df, col_name, min_occurrence=3):
    embeddings = model.encode(df[col_name].values)

    value_counts = df[col_name].value_counts()
    few_occurrences = value_counts[value_counts < min_occurrence].index
    populated_categories = value_counts[value_counts >= min_occurrence].index
    populated_embeddings = model.encode(populated_categories)

    reallocation_dict = {}
    
    for industry in few_occurrences:
        industry_embedding = model.encode([industry])
        similarities = compute_cosine_similarity(populated_embeddings, industry_embedding)
        closest_idx = similarities.argmax()
        closest_category = populated_categories[closest_idx]
        reallocation_dict[industry] = closest_category

    df[col_name] = df[col_name].replace(reallocation_dict)
    return df, reallocation_dict

df_clean, reallocations = reallocate_with_AI(df_clean, 'Industry Category')

print(reallocations)


{'Online education': 'Adult education', 'Professional Association ': 'Association', 'Purchasing': ' Buyer', 'Analytics': 'Information ', 'Cleaning': 'Maintenance', 'Conservation': 'Environmental', 'Plumbing': 'Maintenance', 'Psychologist': 'Scientist', 'Customer service/publishing-adjacent ': 'Customer Service', 'Security and manufacturing company': 'IT Security', 'Wine Importing/Distribution': 'Beverage Production', 'Library--public': 'Archives/Libraries', 'Government Research': 'Government ', 'Fast Food': 'Food', 'Internet': 'Telecommunications', 'CPG': 'Medical Communications', 'Counseling': 'Mental Health', 'Horticulture': 'Agriculture or Forestry', 'Auction House': ' Buyer', 'Education start-up': 'Early Childhood Education', 'Concrete': 'Construction', 'Costruction admin': 'Administration', 'Virtual Assisting ': 'Digital', 'Enviromental ': 'Environment', 'CALL CENTER': 'Customer Service', 'Public Health, local government': 'Health care', 'Think tank': 'Energy - Oil and Gas', 'Tabl

In [18]:
# Display the number of distinct values in each column
distinct_values_count = df_clean.nunique()

print(distinct_values_count)

Timestamp                  27966
Age Group                      7
Industry                    1185
Job Title                  14302
Annual Salary               3654
Additional Compensation      844
Currency                      11
Country                       97
State                        134
City                        4798
Total Experience               8
Field Experience               8
Education Level                6
Gender                         5
Race                          50
Industry Category            162
dtype: int64


### Column Currency
Count how many rows have 'Other' as the currency and remove them

In [19]:
other_currency_count = df_clean[df_clean['Currency'] == 'Other'].shape[0]
print(f"Number of rows with 'Other' currency: {other_currency_count}")

df_clean = df_clean[df_clean['Currency'] != 'Other']

Number of rows with 'Other' currency: 158


### Converting all salary values to USD
New column 'Annual Salary USD' added, using this rates:

'USD': 1,
'CAD': 1.38,
'CHF': 0.87,
'EUR': 0.92,
'GBP': 0.77,
'HKD': 7.77,
'JPY': 149.49,
'SEK': 10.53,
'ZAR': 17.61,
'AUD/NZD': 1.49

In [20]:
# Define conversion rates
conversion_rates = {
    'USD': 1,
    'CAD': 1.38,
    'CHF': 0.87,
    'EUR': 0.92,
    'GBP': 0.77,
    'HKD': 7.77,
    'JPY': 149.49,
    'SEK': 10.53,
    'ZAR': 17.61,
    'AUD/NZD': 1.49
}

# Function to convert salary to USD
def convert_to_usd(row):
    currency = row['Currency']
    annual_salary = row['Annual Salary']
    
    # Ensure the salary is valid (not null or zero)
    if pd.isnull(annual_salary) or annual_salary <= 0:
        return annual_salary  # Leave as is if salary is invalid
    
    # Convert salary if currency exists in the conversion rates
    if currency in conversion_rates:
        return annual_salary / conversion_rates[currency]
    else:
        return annual_salary  # No conversion if currency not in the dictionary

# Apply the conversion to the 'Annual Salary' column
df_clean['Annual Salary USD'] = df_clean.apply(convert_to_usd, axis=1)

# Filter the DataFrame to show only rows where the currency is not 'USD'
non_usd_df = df_clean[df_clean['Currency'] != 'USD']

# Print the first 10 samples with currency not equal to 'USD'
print(non_usd_df[['Industry Category', 'Annual Salary', 'Currency', 'Annual Salary USD']].head(15))

   Industry Category              Annual Salary Currency  Annual Salary USD
1              Computing or Tech   54600         GBP      70909.090909     
14                   Health care   32000         CAD      23188.405797     
15            Telecommunications   24000         GBP      31168.831169     
22  Nonprofit - legal department   63000         CAD      45652.173913     
41             Digital Marketing   35000         GBP      45454.545455     
48  Engineering or Manufacturing  120000         CAD      86956.521739     
49                        Retail   97500         CAD      70652.173913     
54                       Digital   52000         CAD      37681.159420     
59                   Government    52000         GBP      67532.467532     
61                   Government    79000         CAD      57246.376812     
63               Adult education   75000         CAD      54347.826087     
72             Health Insurance    45000         GBP      58441.558442     
81          

### Remove Outliers by Industry Category

In the dataset, salary data is categorized by "Industry Category." Some categories may have extreme values, or outliers, which can distort the analysis. Outliers are values that deviate significantly from the typical salary range for a particular industry.

#### Why Outliers Matter:
- **Skewed Analysis**: Outliers can cause mean salary values to be higher or lower than typical industry salaries.
- **Misleading Insights**: Executive salaries or data errors can make it seem like certain industries pay more or less than they actually do for most jobs.

#### Process in the Code:
1. **Industry-Specific Filtering**: Outliers are detected within each "Industry Category" to ensure the filtering is appropriate for each industry’s salary range.
2. **IQR Calculation**: For each industry, the upper bound is calculated based on the IQR, removing salaries beyond this threshold.
3. **Results**: The cleaned dataset gives a more accurate representation of typical salaries in each industry, without the distortion of extreme values. Number of rows removed: 1021.

In [21]:
def remove_upper_outliers_by_category(df, column, group_column):
    """
    Removes rows where values in a column exceed the upper bound based on IQR within each group.
    
    Args:
        df (pd.DataFrame): The original dataframe.
        column (str): The name of the column for outlier detection (e.g., 'Annual Salary USD').
        group_column (str): The column used for grouping (e.g., 'Industry Category').
    
    Returns:
        pd.DataFrame: A dataframe with high outliers removed.
    """
    def upper_bound_outliers(group):
        Q3 = group[column].quantile(0.75)
        IQR = Q3 - group[column].quantile(0.25)
        upper_bound = Q3 + 1.5 * IQR
        return group[group[column] <= upper_bound]
    
    # Apply the upper bound filter function to each group
    df_filtered = df.groupby(group_column, group_keys=False).apply(upper_bound_outliers)
    
    return df_filtered

# Apply the function to remove high outliers by 'Industry Category'
df_clean_filtered = remove_upper_outliers_by_category(df_clean, 'Annual Salary USD', 'Industry Category')

# Number of rows removed
removed_rows = len(df_clean) - len(df_clean_filtered)
print(f"Number of rows removed: {removed_rows}")

Number of rows removed: 1021
